In [2]:
import lighthouse as lh

import numpy as np
import matplotlib.pyplot as plt 

import torch 

In [ ]:
lh.isochrone.get_isochrones.get_mist_isochrones()

Compiling MIST data
/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m2.00_afe_p0.0_vvcrit0.0_basic.iso -4.0 0
/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.75_afe_p0.0_vvcrit0.0_basic.iso -3.5 1
/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.50_afe_p0.0_vvcrit0.0_basic.iso -3.0 2
/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m3.50_afe_p0.0_vvcrit0.0_basic.iso -2.5 3
/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_basic.iso -2.0 4
/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_basic.iso -1.75 5


In [ ]:
import read_mist_models

ref_iso_solar = read_mist_models.ISO('/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_basic.iso')
ref_iso_mp    = read_mist_models.ISO('/Users/alexa/Light-House/lighthouse/data/MIST/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.50_afe_p0.0_vvcrit0.0_basic.iso')

I = lh.isochrone.MIST()


In [ ]:


ax1 = plt.subplot(121)
ax2 = plt.subplot(122)




age_ind = ref_iso_solar.age_index(10.) #returns the index for the desired age
Teff = 10**ref_iso_solar.isos[age_ind]['log_Teff']
log_g = ref_iso_solar.isos[age_ind]['log_g']
ax1.plot(Teff, log_g, color='k', lw=3, label='read_mist_models') 

print(len(log_g))

mask = ((I.metallicities == 0.0))
for metallicity in I.metallicities[mask]:
    iso = I.get_isochrone(torch.tensor(metallicity, dtype = torch.float64), 
                          torch.tensor(10.,dtype = torch.float64))  
    ax1.plot(iso["Teff"], iso["log_g"], label='Light-House', color='r', lw=3)

print(len(iso["log_g"]))

ax1.legend()
ax1.set_xlabel('Teff)')
ax1.set_ylabel('log(g)')



age_ind = ref_iso_mp.age_index(10.) #returns the index for the desired age
Teff = 10**ref_iso_mp.isos[age_ind]['log_Teff']
log_g = ref_iso_mp.isos[age_ind]['log_g']
ax2.plot(Teff, log_g, color='k', lw=3, label='read_mist_models') 

print(len(log_g))


mask = ((I.metallicities == -0.5))
for metallicity in I.metallicities[mask]:
    iso = I.get_isochrone(torch.tensor(metallicity, dtype = torch.float64), 
                          torch.tensor(10.,dtype = torch.float64))  
    ax2.plot(iso["Teff"], iso["log_g"], label='Light-House', color='r', lw=3)

ax2.legend()
ax2.set_xlabel('Teff)')
ax2.set_ylabel('log(g)')

plt.tight_layout()

In [ ]:
print('ages: ', [round(x,2) for x in ref_iso_solar.ages])

print(ref_iso_solar.num_ages)
print(ref_iso_mp.num_ages)

In [ ]:
print('ages: ', [round(x.item(),2) for x in I.ages])

print(len(I.ages))


print('FeH: ', [round(x.item(),2) for x in I.metallicities])


In [ ]:
def test_isochrone_selection(ref_feh, I):
    metallicity_index = torch.clamp(torch.sum(I.metallicities < ref_feh) - 1, 0)
    print(I.metallicities[metallicity_index])

    metallicity_index_try2 = (I.metallicities  == ref_feh).nonzero(as_tuple=False).squeeze()
    print(I.metallicities[metallicity_index_try2])

testing = test_isochrone_selection(torch.tensor(0.0), I)
testing = test_isochrone_selection(torch.tensor(-0.5), I)


In [ ]:
def test_isochrone_selection(ref_age, I):
    age_index = torch.clamp(torch.sum(I.ages < ref_age) - 1, 0)
    print(I.ages[age_index])

    age_index_try2 = (I.ages  == ref_age).nonzero(as_tuple=False).squeeze()
    print(I.ages[age_index_try2])

testing = test_isochrone_selection(torch.tensor(10.0), I)
testing = test_isochrone_selection(torch.tensor(5.0), I)

testing = test_isochrone_selection(torch.tensor(10.1), I)
testing = test_isochrone_selection(torch.tensor(10.2), I)
testing = test_isochrone_selection(torch.tensor(10.3), I)



In [ ]:
def all_together_now(ref_feh, ref_age, I):
    metallicity_index = (I.metallicities  == ref_feh).nonzero(as_tuple=False).squeeze()
    print(I.metallicities[metallicity_index])
    
    age_index = (I.ages  == ref_age).nonzero(as_tuple=False).squeeze()
    print(I.ages[age_index])

    isochrone = I.isochrone_grid[metallicity_index, age_index].clone()
    isochrone = isochrone[:,isochrone[0] > -999]

    return dict((p, isochrone[i]) for i, p in enumerate(I.param_order))

testing = all_together_now(torch.tensor(0.0), torch.tensor(10.0), I)


In [ ]:

ax1 = plt.subplot(121)
ax2 = plt.subplot(122)


age_ind = ref_iso_solar.age_index(10.) #returns the index for the desired age
Teff = 10**ref_iso_solar.isos[age_ind]['log_Teff']
log_g = ref_iso_solar.isos[age_ind]['log_g']
ax1.plot(Teff, log_g, color='k', lw=3, label='read_mist_models') 

print(len(log_g))

mask = ((I.metallicities == 0.0))
for metallicity in I.metallicities[mask]:
    iso = I.get_isochrone(torch.tensor(metallicity, dtype = torch.float64), 
                          torch.tensor(10.,dtype = torch.float64))  
    ax1.plot(iso["Teff"], iso["log_g"], label='Light-House', color='r', lw=3)

print(len(iso["log_g"]))


testing = all_together_now(torch.tensor(0.0), torch.tensor(10.0), I)
ax1.plot(testing["Teff"], testing["log_g"], label='try again', color='b', lw=1)

print(len(testing["log_g"]))

ax1.legend()
ax1.set_xlabel('Teff)')
ax1.set_ylabel('log(g)')



# age_ind = ref_iso_mp.age_index(10.) #returns the index for the desired age
# Teff = 10**ref_iso_mp.isos[age_ind]['log_Teff']
# log_g = ref_iso_mp.isos[age_ind]['log_g']
# ax2.plot(Teff, log_g, color='k', lw=3, label='read_mist_models') 

# print(len(log_g))


# mask = ((I.metallicities == -0.5))
# for metallicity in I.metallicities[mask]:
#     iso = I.get_isochrone(torch.tensor(metallicity, dtype = torch.float64), 
#                           torch.tensor(10.,dtype = torch.float64))  
#     ax2.plot(iso["Teff"], iso["log_g"], label='Light-House', color='r', lw=3)


# testing = all_together_now(torch.tensor(-0.5), torch.tensor(10.0), I)
# ax2.plot(testing["Teff"], testing["log_g"], label='try again', color='b', lw=1)

# print(len(testing["log_g"]))

ax2.legend()
ax2.set_xlabel('Teff)')
ax2.set_ylabel('log(g)')

plt.tight_layout()